In [1]:
import pandas as pd
import numpy as np

C:\Users\Тимофей\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('houses prices train.csv')

In [5]:
def preprocess(df):
    
    df = df.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis = 1)
    
    df["MasVnrArea"] = df['MasVnrArea'].mean()
    df["LotFrontage"] = df['LotFrontage'].mean()
    df["GarageYrBlt"] = df["GarageYrBlt"].fillna(0)
    
    df['GarageType'] = df['GarageType'].fillna('NA')
    df['GarageFinish'] = df['GarageFinish'].fillna('NA')
    df['GarageCond'] = df['GarageCond'].fillna('NA')
    df['GarageQual'] = df['GarageQual'].fillna('NA')

    df['BsmtQual'] = df['BsmtQual'].fillna('NA')
    df['BsmtCond'] = df['BsmtCond'].fillna('NA')
    df['BsmtExposure'] = df['BsmtExposure'].fillna('NA')
    df['BsmtFinType1'] = df['BsmtFinType1'].fillna('NA')
    df['BsmtFinType2'] = df['BsmtFinType2'].fillna('NA')

    df['Electrical'] = df['Electrical'].fillna('NA')

    df['FireplaceQu'] = df['FireplaceQu'].fillna('NA')
    df['MasVnrType'] = df['MasVnrType'].fillna('NA')
    
    df['HouseAge'] = df['YrSold'] - df['YearBuilt']
    df['HouseRemodelAge'] = df['YrSold'] - df['YearRemodAdd']
    df['TotalBaths'] = df['BsmtFullBath'] + df['FullBath'] + 0.5 * (df['BsmtHalfBath'] + df['HalfBath']) 
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['TotalPorchSF']= (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])
    df['hasgarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    df['haspool'] = df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    
    df = df.drop(['YrSold', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'FullBath', 'BsmtHalfBath', 'HalfBath',
             'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch',
             'WoodDeckSF', 'GarageCars', 'GarageArea', 'PoolArea'], axis = 1)
    
    label_encoding_map = {}

# Список фіч для Label Encoding
    features_label_encoding = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                           'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 
                           'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 
                           'GarageCond']


    def label_encode(feature):
        unique_categories = df[feature].unique()  
        encoding_map = {category: idx for idx, category in enumerate(unique_categories)}  
        df[feature] = df[feature].map(encoding_map)  
        label_encoding_map[feature] = encoding_map  


    for feature in features_label_encoding:
        label_encode(feature)
        
    df = pd.get_dummies(df, columns=['MSZoning', 'Street', 'Utilities', 'LotShape', 'LandContour', 
                                 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 
                                 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 
                                 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                                 'Foundation', 'Heating', 'CentralAir', 'Electrical', 
                                 'GarageType', 'PavedDrive', 
                                 'SaleType', 'SaleCondition'], drop_first= True)
    
    import json
    with open('medians22_04.json', 'r') as file:
        loaded_medians = json.load(file)
        
    cols_with_nans = df.columns[df.isnull().any()].tolist()
    for col in cols_with_nans:
        df[col] = df[col].fillna(loaded_medians[col])
        
    return df

In [6]:
df_preprocessed = preprocess(df)

In [7]:
df_preprocessed

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,70.049958,8450,7,5,103.685262,0,0,0,...,False,False,False,False,True,False,False,False,True,False
1,2,20,70.049958,9600,6,8,103.685262,1,0,0,...,False,False,False,False,True,False,False,False,True,False
2,3,60,70.049958,11250,7,5,103.685262,0,0,0,...,False,False,False,False,True,False,False,False,True,False
3,4,70,70.049958,9550,7,5,103.685262,1,0,1,...,False,False,False,False,True,False,False,False,False,False
4,5,60,70.049958,14260,8,5,103.685262,0,0,0,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,70.049958,7917,6,5,103.685262,1,0,0,...,False,False,False,False,True,False,False,False,True,False
1456,1457,20,70.049958,13175,6,6,103.685262,1,0,0,...,False,False,False,False,True,False,False,False,True,False
1457,1458,70,70.049958,9042,7,9,103.685262,2,1,1,...,False,False,False,False,True,False,False,False,True,False
1458,1459,20,70.049958,9717,5,6,103.685262,1,0,1,...,False,False,False,False,True,False,False,False,True,False


In [8]:
df1 = pd.read_csv('house prices testing.csv')

In [9]:
df1_preprocessed = preprocess(df1)

In [10]:
df1_preprocessed.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,68.580357,11622,5,6,100.709141,0,0,0,...,False,False,False,False,True,False,False,False,True,False
1,1462,20,68.580357,14267,6,6,100.709141,0,0,0,...,False,False,False,False,True,False,False,False,True,False
2,1463,60,68.580357,13830,5,5,100.709141,0,0,1,...,False,False,False,False,True,False,False,False,True,False
3,1464,60,68.580357,9978,6,6,100.709141,0,0,0,...,False,False,False,False,True,False,False,False,True,False
4,1465,120,68.580357,5005,8,5,100.709141,1,0,1,...,False,False,False,False,True,False,False,False,True,False


## Dealing with the following problem

In [11]:
import json

train_columns = df_preprocessed.columns.tolist()
test_columns = df1_preprocessed.columns.tolist()

missing_columns = [col for col in train_columns if col not in test_columns]
for column in missing_columns:
    df1_preprocessed[column] = 0

df1_preprocessed = df1_preprocessed[train_columns]

with open('columns.json', 'w') as json_file:
    json.dump(train_columns, json_file)

In [12]:
df1_preprocessed.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,68.580357,11622,5,6,100.709141,0,0,0,...,False,False,False,False,True,False,False,False,True,False
1,1462,20,68.580357,14267,6,6,100.709141,0,0,0,...,False,False,False,False,True,False,False,False,True,False
2,1463,60,68.580357,13830,5,5,100.709141,0,0,1,...,False,False,False,False,True,False,False,False,True,False
3,1464,60,68.580357,9978,6,6,100.709141,0,0,0,...,False,False,False,False,True,False,False,False,True,False
4,1465,120,68.580357,5005,8,5,100.709141,1,0,1,...,False,False,False,False,True,False,False,False,True,False


In [13]:
df1_preprocessed = df1_preprocessed.drop(['SalePrice'], axis = 1)

In [14]:
df_preprocessed.to_csv('DF_TRAINED_PREPROCESSED_RANDOM_FOREST_2.csv')
df1_preprocessed.to_csv('DF_FOR_PREDICTION_PREPROCESSED_RANDOM_FOREST_2.csv')